Yes, this code is very messy.

In [123]:
# imports
from __future__ import print_function # I got this code from somewhere (I think a Stack Overflow question?) that was apparently using Python 2
import librosa
import numpy as np
import math

# Load local audio file
y, sr = librosa.load('path/to/trimmed/same/header.wav')

First, we find the locations where the waveform changes sign.

The code refers to these as "peaks" because it was written differently before.

In [129]:
peaks_only = []
for i in range(np.shape(y)[0] - 1):
    if y[i] == 0 or y[i + 1] == 0:
        peaks_only.append(False)
    elif np.sign(y[i]) != np.sign(y[i + 1]):
        peaks_only.append(True)
    else:
        peaks_only.append(False)

We calculate these locations relative to the bit period.

In [130]:
peaks = []
is_in_peak = False
for i in range(np.shape(y)[0] - 1):
    if peaks_only[i]:
        if not is_in_peak:
            is_in_peak = True
            peaks.append(i / (0.001927437642 * sr))
    else:
        is_in_peak = False

print(peaks[len(peaks) - 1])

471.8117646381635


Now, we find the time between each of these changes, effectively giving us the period/frequency of the waveform.

In [131]:
margin = 0.2
period = []
for i in range(472):
    current = []
    for j in range(len(peaks)):
        if j + 1 >= len(peaks):
            break
        if peaks[j] >= i + margin:
            current.append(peaks[j + 1] - peaks[j])
        if peaks[j + 1] >= i + 1 - margin:
            break
    period.append(sum(current) / len(current))
print(len(period))

472


Let's check the periods. If we get two discrete values, we did things correctly. (This is where I found out about that precision issue.)

In [132]:
period

[0.16470588232930103,
 0.12235294115890935,
 0.16470588232930097,
 0.16470588232930114,
 0.16470588232930114,
 0.12235294115890945,
 0.12705882351117506,
 0.16470588232930114,
 0.12705882351117523,
 0.12235294115890945,
 0.16470588232930083,
 0.16470588232930083,
 0.16470588232930142,
 0.12235294115890945,
 0.12705882351117523,
 0.16470588232930142,
 0.12705882351117523,
 0.16470588232930083,
 0.16470588232930083,
 0.16470588232930083,
 0.16470588232930083,
 0.12235294115890909,
 0.12705882351117523,
 0.16470588232930203,
 0.12705882351117523,
 0.1223529411589098,
 0.16470588232930083,
 0.16470588232930083,
 0.16470588232930203,
 0.1223529411589098,
 0.12705882351117523,
 0.16470588232930083,
 0.16470588232930083,
 0.16470588232930083,
 0.12705882351117453,
 0.16470588232930083,
 0.12705882351117453,
 0.1223529411589098,
 0.12705882351117453,
 0.16470588232930083,
 0.16470588232930083,
 0.1223529411589098,
 0.12705882351117595,
 0.16470588232930083,
 0.16470588232930083,
 0.12235294115

Finally, let's get the result in binary by looping over all the bit periods. We'll then reverse each byte to correct the endianness.

In [111]:
threshold = 0.15
result = "".join(["0" if x > threshold else "1" for x in period])

# Reverse every byte
result = result[::-1]

In [112]:
result

'0111110101100111010000110111100001010000011000110101001001101011010100010100011101101110011010110100010101100110011001000101101001101010011001010100101101000010010010010100101000110000010111110110001101100011011001100101111101100101011010000111010001011111011110010110001001011111011001000110010101101110011010010110011001011111011101000110010101100111010111110110111101110100010111110110010101101101011010010111010001111011011001100111010001100011011000010110001101100010'

Pop this into CyberChef, and we have the flag!